In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,58,2024-02-25,1,121.842105,100.701754,0.573088,11.010526,21.878947,0.222035,120.322807,...,10.107143,24.833929,0.243821,119.650000,0,2,6,2,17,2024
1,59,2024-02-25,1,116.982759,100.322414,0.559655,11.837931,19.755172,0.217483,115.720690,...,12.642105,24.654386,0.241772,119.356140,0,2,6,2,14,2024
2,57,2024-02-25,7,118.732143,99.692857,0.563339,10.689286,22.296429,0.201482,119.092857,...,11.308772,23.428070,0.179246,121.733333,0,2,6,3,7,2024
3,56,2024-02-25,-2,114.454545,97.243636,0.558491,11.607273,24.614545,0.186436,117.320000,...,11.807143,20.076786,0.175893,111.439286,0,2,6,2,6,2024
4,58,2024-02-25,2,114.456140,96.119298,0.557632,10.740351,26.198246,0.177684,119.222807,...,11.965455,27.612727,0.189909,119.354545,0,2,6,2,8,2024
5,57,2024-02-25,4,121.428571,99.037500,0.580536,10.564286,20.539286,0.207071,121.725000,...,11.130357,24.546429,0.206125,114.085714,0,2,6,2,21,2024
6,57,2024-02-25,-2,111.803571,95.446429,0.534786,10.703571,24.453571,0.187482,115.501786,...,11.310526,25.654386,0.210719,118.498246,0,2,6,3,4,2024
7,58,2024-02-25,3,111.807018,96.842105,0.538754,12.357895,26.198246,0.225667,114.743860,...,10.837500,27.876786,0.218875,119.546429,0,2,6,1,22,2024
8,58,2024-02-25,-3,112.105263,100.943860,0.528351,12.414035,22.654386,0.173158,110.763158,...,13.066667,29.324561,0.212754,117.642105,0,2,6,2,27,2024
9,57,2024-02-25,-1,108.321429,97.551786,0.527786,11.853571,22.216071,0.168482,110.723214,...,12.378182,26.176364,0.197418,110.290909,0,2,6,2,5,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-25,MIL,PHI,1.458716,2.800000,0
1,2024-02-25,LAL,PHO,2.450000,1.571429,0
2,2024-02-25,DAL,IND,1.833333,2.000000,0
3,2024-02-25,CLE,WAS,1.153846,5.700000,1
4,2024-02-25,DEN,GSW,1.892857,1.925926,1
5,2024-02-25,OKC,HOU,1.420168,2.950000,1
6,2024-02-25,CHI,NOP,2.360000,1.617284,0
7,2024-02-25,ORL,ATL,1.869565,1.952381,0
8,2024-02-25,SAS,UTA,3.200000,1.370370,0
9,2024-02-25,CHO,POR,2.240000,1.675676,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-25,MIL,PHI,1.458716,2.800000,0
1,2024-02-25,LAL,PHO,2.450000,1.571429,0
2,2024-02-25,DAL,IND,1.833333,2.000000,0
3,2024-02-25,CLE,WAS,1.153846,5.700000,1
4,2024-02-25,DEN,GSW,1.892857,1.925926,0
5,2024-02-25,OKC,HOU,1.420168,2.950000,1
6,2024-02-25,CHI,NOP,2.360000,1.617284,0
7,2024-02-25,ORL,ATL,1.869565,1.952381,0
8,2024-02-25,SAS,UTA,3.200000,1.370370,0
9,2024-02-25,CHO,POR,2.240000,1.675676,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-25,MIL,PHI,1.458716,2.800000,1
1,2024-02-25,LAL,PHO,2.450000,1.571429,0
2,2024-02-25,DAL,IND,1.833333,2.000000,0
3,2024-02-25,CLE,WAS,1.153846,5.700000,1
4,2024-02-25,DEN,GSW,1.892857,1.925926,0
5,2024-02-25,OKC,HOU,1.420168,2.950000,1
6,2024-02-25,CHI,NOP,2.360000,1.617284,0
7,2024-02-25,ORL,ATL,1.869565,1.952381,0
8,2024-02-25,SAS,UTA,3.200000,1.370370,0
9,2024-02-25,CHO,POR,2.240000,1.675676,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-25,MIL,PHI,1.458716,2.800000,0
1,2024-02-25,LAL,PHO,2.450000,1.571429,0
2,2024-02-25,DAL,IND,1.833333,2.000000,0
3,2024-02-25,CLE,WAS,1.153846,5.700000,1
4,2024-02-25,DEN,GSW,1.892857,1.925926,0
5,2024-02-25,OKC,HOU,1.420168,2.950000,1
6,2024-02-25,CHI,NOP,2.360000,1.617284,0
7,2024-02-25,ORL,ATL,1.869565,1.952381,0
8,2024-02-25,SAS,UTA,3.200000,1.370370,0
9,2024-02-25,CHO,POR,2.240000,1.675676,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-25,MIL,PHI,1.458716,2.800000,1.0
1,2024-02-25,LAL,PHO,2.450000,1.571429,1.0
2,2024-02-25,DAL,IND,1.833333,2.000000,1.0
3,2024-02-25,CLE,WAS,1.153846,5.700000,1.0
4,2024-02-25,DEN,GSW,1.892857,1.925926,1.0
5,2024-02-25,OKC,HOU,1.420168,2.950000,1.0
6,2024-02-25,CHI,NOP,2.360000,1.617284,0.0
7,2024-02-25,ORL,ATL,1.869565,1.952381,0.0
8,2024-02-25,SAS,UTA,3.200000,1.370370,0.0
9,2024-02-25,CHO,POR,2.240000,1.675676,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-25,MIL,PHI,1.458716,2.800000,1.0
1,2024-02-25,LAL,PHO,2.450000,1.571429,1.0
2,2024-02-25,DAL,IND,1.833333,2.000000,1.0
3,2024-02-25,CLE,WAS,1.153846,5.700000,1.0
4,2024-02-25,DEN,GSW,1.892857,1.925926,1.0
5,2024-02-25,OKC,HOU,1.420168,2.950000,1.0
6,2024-02-25,CHI,NOP,2.360000,1.617284,0.0
7,2024-02-25,ORL,ATL,1.869565,1.952381,0.0
8,2024-02-25,SAS,UTA,3.200000,1.370370,0.0
9,2024-02-25,CHO,POR,2.240000,1.675676,1.0
